**Import Data**

In [ ]:
df0 = pd.read_csv("penguins_size.csv")
df = df0.copy()

**Basic Insights**

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()

**Duplicate Check**

In [ ]:
# Veri setindeki yinelenen gözlemleri kontrol eder ve bunları kaldırır

def duplicate_values(df):
    print("Duplicate check...")
    num_duplicates = df.duplicated(subset=None, keep='first').sum()
    if num_duplicates > 0:
        print("There are", num_duplicates, "duplicated observations in the dataset.")
        df.drop_duplicates(keep='first', inplace=True)
        print(num_duplicates, "duplicates were dropped!")
        print("No more duplicate rows!")
    else:
        print("There are no duplicated observations in the dataset.")

In [ ]:
df[df.duplicated(keep=False)].sort_values(by=list(df.columns))

**Null Values**

In [ ]:
def missing_values(df):
    missing_number = df.isnull().sum().sort_values(ascending = False)
    missing_percent = (df.isnull().sum() / df.isnull().count()).sort_values(ascending = False)
    missing_values = pd.concat([missing_number, missing_percent], axis = 1, keys = ['Missing_Number', 'Missing_Percent'])
    return missing_values[missing_values['Missing_Number'] > 0]

In [ ]:
# Veri setindeki tüm '?' değerlerini NaN ile değiştirir

df.replace(to_replace='?',value=np.nan,inplace=True)

#df=df.applymap(lambda x: np.nan if x=='?' else x)

In [ ]:

df['occupation'] = df['occupation'].fillna(method='bfill')

# bfill (backward fill), eksik değerleri bir sonraki mevcut değerle doldurur.
# ffill (forward fill), eksik değerleri bir önceki mevcut değerle doldurur.

In [ ]:
df = df.dropna() #1
df.drop(columns=["Car_Name", "Year"], inplace=True) #2
df.drop(index=[2614], inplace =True)#3

**Categorical Variables**

In [ ]:

df.select_dtypes(include ="object").head()

In [ ]:
df["Clicked on Ad"].value_counts()

In [ ]:
df[df.make_model=="Audi A2"]

In [ ]:
for col in df.select_dtypes('object'):
    print(f"{col:<20}:", df[col].nunique())

# <20 ile en soldan ":" işaretine kadar 20 karakterlik boşluk bırakılır ve feature isimleri bu boşluğa yazdırılır.
# ":" işareti tüm satırlarda aynı hizaya getirilmiş olur.

In [ ]:
for feature in df.columns:
    if df[feature].dtype=="object":
        print(feature, df[feature].nunique())

In [ ]:
plt.figure(figsize = (10,6))
ax = sns.countplot(y = df['occupation'], hue = df['income'])
plt.title("Income by occupation", fontsize = 16)
ax.bar_label(ax.containers[0]);

In [ ]:
df.Sex = df.Sex.replace({'female': 0, 'male': 1})
df['income']=df['income'].map({'<=50K': 0, '>50K': 1})
df['education'].replace(['11th', '9th', '7th-8th', '5th-6th', '10th', '1st-4th', '12th'], 'School', inplace = True)

In [ ]:
cat_features = df.select_dtypes(include='object').columns
num_features = df.select_dtypes(include='number').columns

In [ ]:
def unique_values(df, columns):
    """Prints unique values and their counts for specific columns in the DataFrame."""

    for column_name in columns:
        print(f"Column: {column_name}\n{'-'*30}")
        unique_vals = df[column_name].unique()
        value_counts = df[column_name].value_counts()
        print(f"Unique Values ({len(unique_vals)}): {unique_vals}\n")
        print(f"Value Counts:\n{value_counts}\n{'='*40}\n")

In [ ]:
unique_values(df, cat_features)

In [ ]:
df[df.species =="Gentoo"].groupby("sex").describe().T

In [ ]:
df.loc[336, "sex"] = "MALE"

In [ ]:
# Kategorik feature ların dağılımını göstermek için;

for column in cat_features:
    plt.figure(figsize=(8, 6))
    ax = sns.countplot(x=column, data=df, palette='viridis')
    plt.title(f'Distribution of Categories {column}')

    # Barlar üzerindeki sayımları otomatik olarak etiketle
    ax.bar_label(ax.containers[0])

    plt.xticks(rotation=90)
    plt.show()

In [ ]:
sns.pairplot(df, hue = "species", palette = "Dark2", corner=True);
sns.pairplot(df,hue="Clicked on Ad", corner=True);

In [ ]:
index = 0
plt.figure(figsize=(10,10))
for feature in df.select_dtypes("number"):
    if feature != "species":
        index += 1
        plt.subplot(2,2,index)
        sns.boxplot(x='species',y=feature,data=df)

# pair plotdan aldığımız insight ları burdan da alabiliyoruz.

**Outliers**

In [ ]:
plt.figure(figsize=(16,6))
sns.boxplot(x="make_model", y="price", data=df, whis=3)
plt.show()

# Aşağıdaki görsellere baktığımızda boxplota göre Audi A3, Opel Astra, Opel insignia, Renault clio için 3 wisker
# baz alınarak outlier olabilecek gözlemleri görebiliyoruz. Kendi datalarınızda
# bu görsellere göre her grup için ayrı wisker değerleri belirleyebilirsiniz.

# IQR hesaplamak için şu adımlar izlenir:

# Veriler küçükten büyüğe sıralanır.
# Verilerin %25'ini ve %75'ini temsil eden ilk ve üçüncü çeyrekler hesaplanır.
# IQR, üçüncü çeyrekten ilk çeyrek çıkarılarak elde edilir.

# Q1 = df.groupby('make_model')['price'].quantile(0.25)
# Q3 = df.groupby('make_model')['price'].quantile(0.75)
# IQR = Q3-Q1
# lower_lim = Q1-1.5*IQR
# upper_lim = Q3+1.5*IQR

In [ ]:
# Seaborn'un boxplot fonksiyonu tüm kategoriler için aynı 'whis' değerini kullanır, bu nedenle
# her bir kategoriyi ayrı ayrı çizmek için matplotlib'in boxplot fonksiyonunu kullanacağız.

whisker_values = {
    'Audi A1': 2.0,
    'Audi A3': 1.5,
    'Opel Astra': 2.0,
    'Opel Corsa': 2.5,
    'Opel Insignia': 3.0,
    'Renault Clio': 2.0,
    'Renault Duster': 1.5,
    'Renault Espace': 3.0
}

plt.figure(figsize=(16,6))

for i, make_model in enumerate(whisker_values.keys()):
    model_data = df[df['make_model'] == make_model]['price']
    plt.boxplot(model_data, positions=[i], whis=whisker_values[make_model], widths=0.5)

plt.xticks(range(len(whisker_values)), whisker_values.keys())
plt.show()

In [ ]:
# 1.5 wisker değerine göre her bir gruba ait outlier olabileceğini değerlendirdiğimiz gözlemleri tespit ediyoruz.

total_outliers = []

for model in df.make_model.unique():

    car_prices = df[df["make_model"]== model]["price"]

    Q1 = car_prices.quantile(0.25)
    Q3 = car_prices.quantile(0.75)
    IQR = Q3-Q1
    lower_lim = Q1-1.5*IQR
    upper_lim = Q3+1.5*IQR

    count_of_outliers = (car_prices[(car_prices < lower_lim) | (car_prices > upper_lim)]).count()

    total_outliers.append(count_of_outliers)

    print(f" The count of outlier for {model:<15} : {count_of_outliers:<5}, \
          The rate of outliers : {(count_of_outliers/len(df[df['make_model']== model])).round(3)}")
print()
print("Total_outliers : ",sum(total_outliers), "The rate of total outliers :", (sum(total_outliers)/len(df)).round(3))

**Correlations**

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df.select_dtypes("number").corr(),annot=True, cmap='viridis')
plt.title("Correlation Matrix")

plt.show()

In [ ]:
corr_by_price = df.corr()["price"].sort_values()[:-1]
corr_by_price

# datamızdaki tüm featurların target ile olan corr.larına bakıyoruz
# targetımız olan price ile corr.larını küçükten büyüğe sıralıyoruz.
# Targetın kendisiyle olan corr.unu görmek istemediğimizden slicelama ([:-1]) yapıp -1 ile targetı ignore ediyoruz.



#---------------------------------

plt.figure(figsize = (20,10))
sns.barplot(x = corr_by_price.index, y = corr_by_price)
plt.xticks(rotation=90)
plt.tight_layout();

# featureler ile target arasındaki corr.ları görselleştiriyoruz.

## Multicollinearity

In [ ]:
df_numeric.corr()[(df_numeric.corr()>= 0.9) & (df_numeric.corr() < 1)].any().any()

# +0.9 ile +1 arasındaki corr. değerleri için multicollinearity kontrolünü bu kod ile yapabiliriz.

**Dependent independent Variable Assignment**

In [ ]:
X= df.drop(columns="income")
y= df.income

**Unbalanced**

In [ ]:
# veri setim unbalanced olduğundan stratify=y kullanıyorum

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=101)

In [ ]:
cat_onehot = ['workclass', 'occupation', 'relationship', 'race', 'sex', 'native.country', 'marital.status']
cat_ordinal = ['education', 'capital_diff']

cat_for_edu = ['Preschool', 'School', 'HS-grad','Some-college', 'Assoc-voc', 'Assoc-acdm','Bachelors', 'Masters', 'Prof-school', 'Doctorate']
cat_for_capdiff = ['Low', 'High']

In [ ]:
# train ve test setinin metriclerini karşılaştırabilmek için fonksiyonumuzu tanımlıyoruz.

def train_val(model, X_train, y_train, X_test, y_test):

    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)

    scores = {
    "train": {
    "R2" : r2_score(y_train, y_train_pred),
    "mae" : mean_absolute_error(y_train, y_train_pred),
    "mse" : mean_squared_error(y_train, y_train_pred),
    "rmse" : np.sqrt(mean_squared_error(y_train, y_train_pred))},

    "test": {
    "R2" : r2_score(y_test, y_pred),
    "mae" : mean_absolute_error(y_test, y_pred),
    "mse" : mean_squared_error(y_test, y_pred),
    "rmse" : np.sqrt(mean_squared_error(y_test, y_pred))}
               }

    return pd.DataFrame(scores)

In [ ]:
def adj_r2(y_test, y_pred, df):
    r2 = r2_score(y_test, y_pred)       # Modelin R2 değerini hesaplar
    n = df.shape[0]                     # Veri setindeki gözlem (örnek) sayısını alır
    p = df.shape[1]-1                   # Bağımsız değişkenlerin (feature ların) sayısını alır
    adj_r2 = 1 - (1-r2)*(n-1)/(n-p-1)   # Adjusted R2 formülünü kullanarak değeri hesaplar
    return adj_r2